# Introduction to torch.autograd

`torch.autograd` is PyTorch's automatic differentiation engine that powers neural network training. In this section, you will get a conceptual understanding of how autograd helps a neural network train.

## Background

Neural networks (NNs) are a collection of nested functions that are executed on some input data. These functions are defined by parameters (consisting of weights and biases), which in PyTorch are stored in tensors.

Training a NN happens in two steps:

**Forward Propagation:** In forward prop, the NN makes its best guess about the correct output. It runs the input data through each of its functions to make this guess.

**Backward Propagation:** In backprop, the NN adjusts its parameters proportionate to the error in its guess. It does this by traversing backwards from the output, collecting the derivatives of the error with respect to the parameters of the functions (gradients), and optimizing the parameters using gradient descent.

## Usage in PyTorch

Let's take a look at a single training step. For this example, we load a pretrained resnet18 model from `torchvision`. We create a random data tensor to represent a single image with 3 channels, and height & width of 64, and its corresponding `label` initialized to some random values. Label in pretrained models has shape (1,1000).

In [1]:
import torch
from torchvision.models import resnet18, ResNet18_Weights

model = resnet18(weights=ResNet18_Weights.DEFAULT)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 136MB/s]


In [3]:
print("Data tensor info: \n")
print(f"Data tensor shape: {data.shape} \n")
print(f"Data tensor type: {data.dtype} \n")

Data tensor info: 

Data tensor shape: torch.Size([1, 3, 64, 64]) 

Data tensor type: torch.float32 



This tensor is 4D.

Here's a breakdown of the dimensions in the shape `torch.Size([1, 3, 64, 64])`:

1. **1**: The first dimension typically represents the *batch size*, indicating that there's one sample in this batch.
2. **3**: The second dimension often represents the *channels*, such as RGB channels in an image.
3. **64**: The third dimension is the *height* of the data (in this case, an image).
4. **64**: The fourth dimension is the *width* of the data (image width).

So, `[1, 3, 64, 64]` represents a batch of one 64x64 RGB image (3 channels), making it a 4D tensor with dimensions `[Batch, Channels, Height, Width]`.

Next, we run the input data through the model through each of its layers to make a prediction. This is the forward pass.



In [8]:
prediction = model(data) # forward pass

We use the model's prediction and the corresponding label to calculate the error (`loss`). The next step is to backpropagate this error through the network. Backward propagation is kicked off when we call `.backward()` on the error tensor. Autograd then calculates and stores the gradients for each model parameter in the parameter's `.grad` attribute.

In [9]:
loss = (prediction - labels).sum()
loss.backward() # backward pass

Next, we load an optimizer, in this case SGD(Stochastic Gradient Descent) with a learning rate of 0.01 and momentum of 0.9. We register all the parameters of the model in the optimizer.



In [10]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

Finally, we call `.step()` to initiate gradient descent. The optimizer adjusts each parameter by its gradient stored in `.grad`.



In [11]:
optim.step() # gradient descent

## Differentiation in Autograd


Let's take a look at how autograd collects gradients. We create two tensors a and b with `requires_grad=True`. This signals to `autograd` that every operation on them should be tracked.

In [12]:
import torch

a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

We create another tensor `Q` from `a` and `b`.

$Q = 3a^3 - b^2$


In [13]:
Q = 3 * a ** 3 - b ** 2

Let's assume `a` and `b` are parameters of an NN, and Q to be the error. In NN training, we want gradients of the error w.r.t. parameters, i.e.


$\frac{\partial Q}{\partial a} = 9a^2$

$\frac{\partial Q}{\partial b} = -2b$


When we call `.backward()` on Q, autograd calculates these gradients and stores them in the respective tensors' `.grad` attribute.

We need to explicitly pass a `gradient` argument in `Q.backward()` because it is a vector. `gradient` is a tensor of the same shape as `Q`, and it represents the gradient of Q w.r.t. itself, i.e.

$\frac{\partial Q}{\partial Q} = 1$


Equivalently, we can also aggregate Q into a scalar and call backward implicitly, like `Q.sum().backward()`.



In [14]:
external_grad = torch.tensor([1, 1])
Q.backward(gradient=external_grad)

Tabii ki!

Burada \( Q \), bir hata fonksiyonu gibi düşünebiliriz ve diyelim ki \( Q \), yapay sinir ağı parametreleri \( a \) ve \( b \) ile hesaplanıyor. Amacımız, hatanın \( a \) ve \( b \) parametrelerine göre türevini (gradyanını) bulmak. Bu da sinir ağı eğitiminde önemli çünkü ağ, hatayı azaltmak için gradyanları kullanarak parametreleri günceller.

### Neden \([1, 1]\) Gradyan Kullanıyoruz?

Eğer \( Q \) bir vektör veya birden fazla elemandan oluşuyorsa, PyTorch'un gradyanı doğru hesaplayabilmesi için başta bir dış gradyan tanımlamamız gerekiyor. Bu dış gradyanı şöyle tanımlıyoruz:

```python
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)
```

#### Açıklaması:
1. **Başlangıç Gradyanı Sağlamak**: Burada \([1, 1]\), \( Q \)'nun her bir bileşenine göre kendisinin gradyanı olarak kabul edilir. Yani, bu $(\frac{dQ}{dQ} = [1, 1])$ demektir. Bu başlangıç gradyanı, parametreler \( a \) ve \( b \)'ye doğru düzgün bir şekilde yayılır ve sonuçta PyTorch, \( Q \)'ya göre \( a \) ve \( b \)'nin gradyanını doğru bir şekilde hesaplayabilir.

2. **Çoklu Bileşenler için Gerekli**: Eğer \( Q \) iki elemanlı bir vektörse (örneğin $( Q = [Q_1, Q_2] )$ gibi), PyTorch hangi bileşene göre hesaplama yapacağını bilmez. Bu durumda, \([1, 1]\) kullanarak her bileşenin bağımsız olduğunu ve kendine göre \(1\) gradyana sahip olduğunu belirtmiş oluruz.

3. **Tekil Skalar Değilse Gerekli**: PyTorch'ta `.backward()` sadece tek bir skalar (tekil sayı) üzerinde çalışabilir. Eğer \( Q \) skalar değil de çok bileşenli bir yapıdaysa, dışarıdan bir gradyan vermezsek PyTorch bunu işlemez. Bu yüzden \([1, 1]\) gibi bir gradyanla her bir bileşeni başlatmamız gerekir.

Bu, PyTorch'un otomatik türev alma sisteminde hatanın doğru bir şekilde geriye yayılmasını sağlıyor, ve sonuç olarak model parametreleri (mesela \( a \) ve \( b \)) doğru şekilde güncelleniyor.

---

Burada asıl mesele **her bir \( Q \) bileşeninin geriye yayılma sırasında ne kadar ağırlıklandırılacağı** ve **inputların gradyanlarının nasıl etkilenmesini istediğimiz** ile ilgilidir.

### Özetle:
1. **Ağırlıklandırma (Weighting)**: \([1, 6]\) gibi bir gradyan vererek, her bileşene farklı önem veya ağırlık verebiliyoruz. Yani, \( Q \)'nun bileşenlerinin katkısının eşit veya farklı olmasını bu sayede kontrol ediyoruz.
2. **Geriye Yayılmada Etki**: Eğer \( Q \)'nun bazı bileşenlerinin model parametrelerine daha fazla etki etmesini istiyorsak, o bileşenlere daha yüksek bir değer atayarak bunu sağlarız. Mesela \([1, 6]\) ile ikinci bileşene daha fazla önem vermiş oluyoruz.
3. **Bağımsızlık Durumu**: Bu ağırlıklandırma, inputların veya bileşenlerin bağımsız olup olmadığını değil, her bir bileşenin gradyan hesabındaki etkisini değiştirmek için kullanılır. Bu etki, geriye yayılma sırasında parametre güncellemelerini etkileyen bir faktördür.

Yani, burada gradyanın ağırlıklandırılması önemli olan nokta ve bu, modelin her bileşene göre farklı hassasiyetlerde güncellenmesini sağlıyor.

Gradients are now deposited in `a.grad` and `b.grad`.





In [18]:
# check if collected gradients are correct

print(9 * a ** 2 == a.grad)

print(-2 * b == b.grad)

tensor([True, True])
tensor([True, True])


### Devamında bir sürü bölüm var ama not alması zor yerler. Link: https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html